# Experiment 8

Experiment 8 aims to examine the stability of different control method with respect to starting position. The experiment will use the same methods as most previous ones however there will be greater examination of end number of links involved for each swarm control method.

In [1]:
## Imports

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scikit_posthocs as sp
import matplotlib.pyplot as plt
import networkx as nx

import logging

from src.simulation import Simulation
from src.network import Network
from src.render import Render

logging.disable(logging.CRITICAL)

In [2]:
network_file_1 = "networks/Net3.inp"
network_file_2 = "networks/250701 K709vs2-Export.inp"
network_file_3 = "networks/Net6.inp"

env1 = Network(network_file_1)
env2 = Network(network_file_2)
env3 = Network(network_file_3)

g_env1 = env1.water_network_model.to_graph().to_undirected()
g_env2 = env2.water_network_model.to_graph().to_undirected()
g_env3 = env3.water_network_model.to_graph().to_undirected()

d_env1 = g_env1.degree
d_env2 = g_env2.degree
d_env3 = g_env3.degree

env1_start_pool = [node for node, degree in d_env1 if degree == 1]
env2_start_pool = [node for node, degree in d_env2 if degree == 1]
env3_start_pool = [node for node, degree in d_env3 if degree == 1]

env1_num_links = env1.graph_num_links
env2_num_links = env2.graph_num_links
env3_num_links = env3.graph_num_links

print("Start Pool - Net3: ", env1_start_pool)
print("Number of Start Nodes - Net3: ", len(env1_start_pool))
print("Number of Nodes - Net3: ", env1.graph_num_nodes)
print("Number of Links - Net3: ", env1.graph_num_links)

print("Start Pool - 250701 K709vs2-Export: ", env2_start_pool)
print("Number of Start Nodes - 250701 K709vs2-Export: ", len(env2_start_pool))
print("Number of Nodes - 250701 K709vs2-Export: ", env2.graph_num_nodes)
print("Number of Links - 250701 K709vs2-Export: ", env2.graph_num_links)

print("Start Pool - Net6: ", env3_start_pool)
print("Number of Start Nodes - Net6: ", len(env3_start_pool))
print("Number of Nodes - Net6: ", env3.graph_num_nodes)
print("Number of Links - Net6: ", env3.graph_num_links)

Start Pool - Net3:  ['15', '35', '131', '166', '167', '203', '219', '225', '231', '243', '253', 'River', 'Lake', '1', '2', '3']
Number of Start Nodes - Net3:  16
Number of Nodes - Net3:  97
Number of Links - Net3:  119
Start Pool - 250701 K709vs2-Export:  ['A2148', 'A3007', 'A3018', 'A5065', 'A5071', 'A5078', 'A5153', 'A5169', 'A5174', 'A5175', 'A5178', 'A5183', 'A5184', 'A5185', 'A5187', 'A5189', 'A5191', 'A5197', 'A5200', 'A5202', 'A5205', 'A5245', 'A5247', 'A5255', 'A5271', 'A5279', 'A5283', 'A5285', 'A5306', 'A5311', 'A5323', 'A5342', 'A5343', 'A5348', 'A5351', 'A5354', 'A5357', 'A5376', 'A5377', 'A5379', 'A5383', 'A5387', 'A5390', 'A5391', 'A5394', 'A5396', 'A5398', 'A5400', 'A5404', 'A5407', 'A5408', 'A5412', 'A5423', 'A5424', 'A5426', 'A5428', 'A5430', 'A5431', 'A5435', 'A5440', 'A5448', 'A5450', 'A5451', 'A5460', 'A5462', 'A5471', 'A5473', 'A5474', 'A5480', 'A5481', 'A5485', 'A5502', 'A5510', 'A5513', 'A5515', 'A5701', 'A5708', 'AN-1871', 'N-0457', 'N-0458', 'N-0459', 'N-0460',

In [3]:
def run_simulation_batch(env, num_agents, start_nodes, filepath, max_turns=100):
    print("Starting Simulation Batch - ", filepath)
    print("Number of Agents: ", num_agents)
    print("Start Nodes: ", start_nodes)
    print("Max Turns: ", max_turns)
    
    # Run the simulations for no swarm control
    path = f'{filepath}/NoSwarm'
    simulations_1 = []
    for node in start_nodes:
        print("Starting No Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=False, start_positions=[node], filepath=path)
        simulations_1.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)
    yield simulations_1
    
    # Run the simulations for naive swarm control
    path = f'{filepath}/NaiveSwarm'
    swarm_config = {'swarm': True, 'swarm_type': 'naive'}
    simulations_2 = []
    for node in start_nodes:
        print("Starting Naive Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=True, swarm_config=swarm_config, start_positions=[node], filepath=path)
        simulations_2.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)
    yield simulations_2
    
    # Run the simulations for informed mean swarm control
    path = f'{filepath}/InformedMeanSwarm'
    swarm_config = {'swarm': True, 'swarm_type': 'informed', 'allocation_threshold': 'mean'}
    simulations_3 = []
    for node in start_nodes:
        print("Starting Informed Mean Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=True, swarm_config=swarm_config, start_positions=[node], filepath=path)
        simulations_3.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)    
    yield simulations_3
    
    # Run the simulations for informed median swarm control
    path = f'{filepath}/InformedMedianSwarm'
    swarm_config = {'swarm': True, 'swarm_type': 'informed', 'allocation_threshold': 'median'}
    simulations_4 = []
    for node in start_nodes:
        print("Starting Informed Median Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=True, swarm_config=swarm_config, start_positions=[node], filepath=path)
        simulations_4.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)
    yield simulations_4

## Simulations

### Simulation 1 - Environment 1

In [4]:
start_nodes = env1_start_pool[:15]
num_agents = 10
max_turns = 100
filepath = "notable-results/Experiment-8/Env1"

print("Start Nodes: ", start_nodes)

Start Nodes:  ['15', '35', '131', '166', '167', '203', '219', '225', '231', '243', '253', 'River', 'Lake', '1', '2']


In [5]:
simulations_8_1 = run_simulation_batch(env1, num_agents, start_nodes, filepath, max_turns=max_turns)

no_swarm_env1 = next(simulations_8_1)
naive_swarm_env1 = next(simulations_8_1)
informed_mean_swarm_env1 = next(simulations_8_1)
informed_median_swarm_env1 = next(simulations_8_1)

Starting Simulation Batch -  notable-results/Experiment-8/Env1
Number of Agents:  10
Start Nodes:  ['15', '35', '131', '166', '167', '203', '219', '225', '231', '243', '253', 'River', 'Lake', '1', '2']
Max Turns:  100
Starting No Swarm Simulation from Start Node:  15
Starting No Swarm Simulation from Start Node:  35
Starting No Swarm Simulation from Start Node:  131
Starting No Swarm Simulation from Start Node:  166
Starting No Swarm Simulation from Start Node:  167
Starting No Swarm Simulation from Start Node:  203
Starting No Swarm Simulation from Start Node:  219
Starting No Swarm Simulation from Start Node:  225
Starting No Swarm Simulation from Start Node:  231
Starting No Swarm Simulation from Start Node:  243
Starting No Swarm Simulation from Start Node:  253
Starting No Swarm Simulation from Start Node:  River
Starting No Swarm Simulation from Start Node:  Lake
Starting No Swarm Simulation from Start Node:  1
Starting No Swarm Simulation from Start Node:  2
Starting Naive Swarm

In [6]:
# Check that the number of simulations is correct
assert len(no_swarm_env1) == len(start_nodes)
assert len(naive_swarm_env1) == len(start_nodes)
assert len(informed_mean_swarm_env1) == len(start_nodes)
assert len(informed_median_swarm_env1) == len(start_nodes)

### Simulation 2 - Environment 2

In [7]:
start_nodes = env2_start_pool[:15]
num_agents = 10
max_turns = 100
filepath = "notable-results/Experiment-8/Env2"

print("Start Nodes: ", start_nodes)

Start Nodes:  ['A2148', 'A3007', 'A3018', 'A5065', 'A5071', 'A5078', 'A5153', 'A5169', 'A5174', 'A5175', 'A5178', 'A5183', 'A5184', 'A5185', 'A5187']


In [8]:
simulations_8_2 = run_simulation_batch(env2, num_agents, start_nodes, filepath, max_turns=max_turns)

no_swarm_env2 = next(simulations_8_2)
naive_swarm_env2 = next(simulations_8_2)
informed_mean_swarm_env2 = next(simulations_8_2)
informed_median_swarm_env2 = next(simulations_8_2)

Starting Simulation Batch -  notable-results/Experiment-8/Env2
Number of Agents:  10
Start Nodes:  ['A2148', 'A3007', 'A3018', 'A5065', 'A5071', 'A5078', 'A5153', 'A5169', 'A5174', 'A5175', 'A5178', 'A5183', 'A5184', 'A5185', 'A5187']
Max Turns:  100
Starting No Swarm Simulation from Start Node:  A2148
Starting No Swarm Simulation from Start Node:  A3007
Starting No Swarm Simulation from Start Node:  A3018
Starting No Swarm Simulation from Start Node:  A5065
Starting No Swarm Simulation from Start Node:  A5071
Starting No Swarm Simulation from Start Node:  A5078
Starting No Swarm Simulation from Start Node:  A5153
Starting No Swarm Simulation from Start Node:  A5169
Starting No Swarm Simulation from Start Node:  A5174
Starting No Swarm Simulation from Start Node:  A5175
Starting No Swarm Simulation from Start Node:  A5178
Starting No Swarm Simulation from Start Node:  A5183
Starting No Swarm Simulation from Start Node:  A5184
Starting No Swarm Simulation from Start Node:  A5185
Startin

In [9]:
# Check that the number of simulations is correct
assert len(no_swarm_env2) == len(start_nodes)
assert len(naive_swarm_env2) == len(start_nodes)
assert len(informed_mean_swarm_env2) == len(start_nodes)
assert len(informed_median_swarm_env2) == len(start_nodes)

### Simulation 3 - Environment 3

In [10]:
start_nodes = env3_start_pool[:15]
num_agents = 10
max_turns = 100
filepath = "notable-results/Experiment-8/Env3"

print("Start Nodes: ", start_nodes)

Start Nodes:  ['JUNCTION-12', 'JUNCTION-16', 'JUNCTION-20', 'JUNCTION-21', 'JUNCTION-38', 'JUNCTION-39', 'JUNCTION-40', 'JUNCTION-41', 'JUNCTION-45', 'JUNCTION-53', 'JUNCTION-74', 'JUNCTION-75', 'JUNCTION-81', 'JUNCTION-88', 'JUNCTION-96']


In [11]:
simulations_8_3 = run_simulation_batch(env3, num_agents, start_nodes, filepath, max_turns=max_turns)

no_swarm_env3 = next(simulations_8_3)
naive_swarm_env3 = next(simulations_8_3)
informed_mean_swarm_env3 = next(simulations_8_3)
informed_median_swarm_env3 = next(simulations_8_3)

Starting Simulation Batch -  notable-results/Experiment-8/Env3
Number of Agents:  10
Start Nodes:  ['JUNCTION-12', 'JUNCTION-16', 'JUNCTION-20', 'JUNCTION-21', 'JUNCTION-38', 'JUNCTION-39', 'JUNCTION-40', 'JUNCTION-41', 'JUNCTION-45', 'JUNCTION-53', 'JUNCTION-74', 'JUNCTION-75', 'JUNCTION-81', 'JUNCTION-88', 'JUNCTION-96']
Max Turns:  100
Starting No Swarm Simulation from Start Node:  JUNCTION-12
Starting No Swarm Simulation from Start Node:  JUNCTION-16
Starting No Swarm Simulation from Start Node:  JUNCTION-20
Starting No Swarm Simulation from Start Node:  JUNCTION-21
Starting No Swarm Simulation from Start Node:  JUNCTION-38
Starting No Swarm Simulation from Start Node:  JUNCTION-39
Starting No Swarm Simulation from Start Node:  JUNCTION-40
Starting No Swarm Simulation from Start Node:  JUNCTION-41
Starting No Swarm Simulation from Start Node:  JUNCTION-45
Starting No Swarm Simulation from Start Node:  JUNCTION-53
Starting No Swarm Simulation from Start Node:  JUNCTION-74
Starting N

In [12]:
# Check that the number of simulations is correct
assert len(no_swarm_env3) == len(start_nodes)
assert len(naive_swarm_env3) == len(start_nodes)
assert len(informed_mean_swarm_env3) == len(start_nodes)
assert len(informed_median_swarm_env3) == len(start_nodes)

## Analysis

In [13]:
from typing import List

# Read the results from the simulations
def read_results(simulations, swarm_type) -> List[pd.DataFrame]:
    results = []
    for sim in simulations:
        start_node, path = sim
        df = pd.read_csv(f"{path}/results.csv")
        df['start_node'] = start_node
        df.start_node = start_node
        df['swarm_type'] = swarm_type
        df.swarm_type = swarm_type
        results.append(df)
    return results

def clean_dataframe(dataframe: pd.DataFrame) -> pd.Series:
    max_row = dataframe.loc[dataframe['abs_links_explored'].idxmax()]
    max_row = max_row[['abs_links_explored', 'start_node', 'swarm_type']]
    return max_row

def sim_dataframe(simulations, swarm_type):
    df = pd.DataFrame()
    results = read_results(simulations, swarm_type)
    for result in results:
        result = clean_dataframe(result)
        result = result.reset_index(drop=True)
        df = pd.concat([df, result], axis=1, ignore_index=True)
    return df

def make_dataframe(simulations, swarm_type):
    df = sim_dataframe(simulations, swarm_type)
    df = df.T
    df.columns = ['abs_links_explored', 'start_node', 'swarm_type']
    df = df.reset_index(drop=True)
    return df

def batch_make_dataframe(simulations:list, swarm_types:list):
    df = pd.DataFrame()
    for i in range(len(simulations)):
        df_ = make_dataframe(simulations[i], swarm_types[i])
        df = pd.concat([df, df_], axis=0, ignore_index=True)
    return df

def melt_pivot(df:pd.DataFrame, swarm_types:list):
    df = df.melt(id_vars=['start_node', 'swarm_type'], var_name='metric', value_name='value')
    df = df.pivot(index=['start_node', 'swarm_type'], columns='metric', values='value')
    df = df.reset_index()
    df = df.rename(columns={'abs_links_explored': 'links_explored'})
    df = df.sort_values(by=['swarm_type', 'start_node'])
    df = df.reset_index(drop=True)
    return df
      
    

In [14]:
sims = [no_swarm_env1, naive_swarm_env1, informed_mean_swarm_env1, informed_median_swarm_env1]
swarm_types = ['no-swarm', 'naive', 'informed-mean', 'informed-median']

df_env1 = batch_make_dataframe(sims, swarm_types)

In [15]:
env1_res = melt_pivot(df_env1, swarm_types)
env1_res.head()

metric,start_node,swarm_type,links_explored
0,1,informed-mean,100
1,131,informed-mean,103
2,15,informed-mean,106
3,166,informed-mean,118
4,167,informed-mean,100


In [16]:
sims = [no_swarm_env2, naive_swarm_env2, informed_mean_swarm_env2, informed_median_swarm_env2]
swarm_types = ['no-swarm', 'naive', 'informed-mean', 'informed-median']

df_env2 = batch_make_dataframe(sims, swarm_types)

In [17]:
env2_res = melt_pivot(df_env2, swarm_types)
env2_res.head()

metric,start_node,swarm_type,links_explored
0,A2148,informed-mean,156
1,A3007,informed-mean,181
2,A3018,informed-mean,104
3,A5065,informed-mean,197
4,A5071,informed-mean,203


In [18]:
sims = [no_swarm_env3, naive_swarm_env3, informed_mean_swarm_env3, informed_median_swarm_env3]
swarm_types = ['no-swarm', 'naive', 'informed-mean', 'informed-median']

df_env3 = batch_make_dataframe(sims, swarm_types)

In [19]:
env3_res = melt_pivot(df_env3, swarm_types)
env3_res.head()

metric,start_node,swarm_type,links_explored
0,JUNCTION-12,informed-mean,270
1,JUNCTION-16,informed-mean,221
2,JUNCTION-20,informed-mean,290
3,JUNCTION-21,informed-mean,290
4,JUNCTION-38,informed-mean,218


In [20]:
def multi_batch_df(dataframes:List[pd.DataFrame]):
    df = pd.DataFrame()
    for dataframe in dataframes:
        df = pd.concat([df, dataframe], axis=0, ignore_index=True)
    return df

In [21]:
results = multi_batch_df([df_env1, df_env2, df_env3])
assert len(results) == 15*3*4, "The number of rows in the dataframe is incorrect"

In [22]:
batch_long = pd.melt(results, id_vars=['start_node', 'swarm_type'], value_vars=['abs_links_explored'])

In [23]:
batch_wide = batch_long.pivot_table(index=['start_node', 'swarm_type'], columns='variable', values='value').reset_index()
batch_wide

variable,start_node,swarm_type,abs_links_explored
0,1,informed-mean,100.0
1,1,informed-median,100.0
2,1,naive,95.0
3,1,no-swarm,16.0
4,131,informed-mean,103.0
...,...,...,...
175,Lake,no-swarm,27.0
176,River,informed-mean,93.0
177,River,informed-median,93.0
178,River,naive,112.0


### Examine stability of the swarm control methods to changes in starting position

#### ANOVA Assumptions Test

In [24]:
# Test for normality of the data using the Shapiro-Wilk test
for col in batch_wide.columns:
    if col not in ['start_node', 'swarm_type']:
        _, p = stats.shapiro(batch_wide[col])
        sig_val = 0.05
        if p < sig_val:
            print(f'{col}: p-value={p:.10f} - The data is not normally distributed')
        else:
            print(f'{col}: p-value={p:.10f} - The data is normally distributed')

abs_links_explored: p-value=0.0000010454 - The data is not normally distributed


#### Descriptive Statistics

It would be nice to compare between the different networks and just use it all together as a large dataset. Unfortunately, this isn't possible. The networks are different enough that the results will be affected too much by the topology of the networks - the connectedness etc. - which renders such comparisons futile.

In [25]:
def describe(dataframe:pd.DataFrame):
    grouped = dataframe.groupby(['swarm_type'])['links_explored']
    means = grouped.mean()
    stds = grouped.std()
    
    cvs = (stds / means) * 100
    
    return means, stds, cvs

def run_describe(dataframe:pd.DataFrame, environment:str):
    means, stds, cvs = describe(dataframe)
    
    print("Descriptive Stats: {}".format(environment))
    print()
    print("Mean number of links explored:")
    print(means)
    print()
    print("Standard deviation of number of links explored:")
    print(stds)
    print()
    print("Coefficient of variation of number of links explored:")
    print(cvs)
    print()
    print("--------------------------------------------------")
    print()
    

In [26]:
run_describe(env1_res, 'Environment 1')

Descriptive Stats: Environment 1

Mean number of links explored:
swarm_type
informed-mean      99.466667
informed-median    99.466667
naive              84.800000
no-swarm           43.266667
Name: links_explored, dtype: float64

Standard deviation of number of links explored:
swarm_type
informed-mean       8.399546
informed-median     8.399546
naive              30.515102
no-swarm           15.993153
Name: links_explored, dtype: float64

Coefficient of variation of number of links explored:
swarm_type
informed-mean       8.444584
informed-median     8.444584
naive              35.984790
no-swarm           36.964145
Name: links_explored, dtype: float64

--------------------------------------------------



In [27]:
run_describe(env2_res, 'Environment 2')

Descriptive Stats: Environment 2

Mean number of links explored:
swarm_type
informed-mean      179.066667
informed-median    179.066667
naive              134.533333
no-swarm            64.266667
Name: links_explored, dtype: float64

Standard deviation of number of links explored:
swarm_type
informed-mean      40.972930
informed-median    40.972930
naive              58.783947
no-swarm           23.986504
Name: links_explored, dtype: float64

Coefficient of variation of number of links explored:
swarm_type
informed-mean      22.881383
informed-median    22.881383
naive              43.694708
no-swarm           37.323399
Name: links_explored, dtype: float64

--------------------------------------------------



In [28]:
run_describe(env3_res, 'Environment 3')

Descriptive Stats: Environment 3

Mean number of links explored:
swarm_type
informed-mean      231.400000
informed-median    231.400000
naive              182.133333
no-swarm            66.666667
Name: links_explored, dtype: float64

Standard deviation of number of links explored:
swarm_type
informed-mean      68.944906
informed-median    68.944906
naive              56.217520
no-swarm           30.499805
Name: links_explored, dtype: float64

Coefficient of variation of number of links explored:
swarm_type
informed-mean      29.794687
informed-median    29.794687
naive              30.866135
no-swarm           45.749707
Name: links_explored, dtype: float64

--------------------------------------------------



: 